In [ ]:
# for Google Colab
# !git clone https://github.com/pavelshapka/Q_func_AM.git
# !pip3 install -r /content/Q_func_AM/requirements.txt
# import os
# os.chdir("/content/Q_func_AM")

In [ ]:
from classification.default_loader import get_base_dataloaders, CIFAR10_MEANS, CIFAR10_STD
from sas_stack.sars_dataloader import get_sars_dataloaders
from sa_stack.sar_dataloader import get_sar_dataloaders

import jax
import matplotlib.pyplot as plt
from classification.trainer import TrainerModule
from classification.inception_v1 import InceptionNetV1
from flax import linen as nn

import numpy as np


%load_ext autoreload
%autoreload 2

ImportError: attempted relative import with no known parent package

In [ ]:
num_steps = 15

train_loader, test_loader = get_sars_dataloaders(batch_size=15,
                                                 reward=10,
                                                 gamma=0.95,
                                                 num_steps=num_steps)

In [ ]:
train_iter = iter(train_loader)
images, rewards = next(train_iter)

In [ ]:
states = images[..., :3]
actions = images[..., 3:6]
states_hat = images[..., 6:9]

In [ ]:
def plot_trajectory(s, a, s_hat, num_steps=num_steps-1):
    plt.figure(figsize=(20, 6))

    for i in range(num_steps):
        # State
        plt.subplot(3, num_steps, i+1)
        plt.imshow((s[i].numpy() * CIFAR10_STD + CIFAR10_MEANS))  # Денормализация
        plt.title(f"State {i}")
        plt.axis('off')

        # Action (визуализируем как разницу)
        plt.subplot(3, num_steps, num_steps+i+1)
        plt.imshow((a[i].numpy() * CIFAR10_STD + CIFAR10_MEANS))
        plt.title(f"Action {i}")
        plt.axis('off')

        # Next State
        plt.subplot(3, num_steps, 2*num_steps+i+1)
        plt.imshow((s_hat[i].numpy() * CIFAR10_STD + CIFAR10_MEANS))
        plt.title(f"Next State {i}")
        plt.axis('off')

    plt.tight_layout()
    plt.show()

# Визуализируем первую траекторию в батче
plot_trajectory(states, actions, states_hat)

## Проверим, что модель корректна через обучение классификации на CIFAR-10

In [ ]:
train_loader, val_loader, test_loader = get_base_dataloaders(batch_size=16)

In [ ]:
train_iter = iter(train_loader)
images, _ = next(train_iter)

In [ ]:
NUM_IMAGES = 10

nrows, ncols = 2, 5
fig, axs = plt.subplots(figsize=(20, 10), nrows=nrows, ncols=ncols)

for i in range(NUM_IMAGES):
    img = (images[i]._numpy() * CIFAR10_STD + CIFAR10_MEANS)
    axs[i//ncols, i%ncols].imshow(img)
    axs[i//ncols, i%ncols].axis("off")

plt.show()
plt.close()

In [ ]:
def train_classifier(*args, num_epochs=200, start_from=0, train=True, **kwargs):
    # Create a trainer module with specified hyperparameters
    trainer = TrainerModule(*args, **kwargs)
    if trainer.checkpoint_exists():
        start_from = trainer.load_model()

    if train:
        print(f"current_epoch = {start_from}")
        trainer.train_model(train_loader,
                            val_loader,
                            rng=jax.random.PRNGKey(42),
                            num_epochs=num_epochs,
                            start_from=start_from)
    # Test trained model
    val_acc = trainer.eval_model(val_loader)
    test_acc = trainer.eval_model(test_loader)
    return trainer, {'val': val_acc, 'test': test_acc}

In [ ]:
googlenet_trainer, googlenet_results = train_classifier(model_class=InceptionNetV1,
                                                        model_name="InceptionNetV1",
                                                        model_hparams={"num_classes": 10,
                                                                       "activation": nn.relu},
                                                        optimizer_name="adamw",
                                                        optimizer_hparams={"lr": 3e-4,
                                                                           "weight_decay": 1e-5},
                                                        exmp_imgs=jax.device_put(next(iter(train_loader))[0]._numpy()),
                                                        num_epochs=10,
                                                        train=True)

In [ ]:
googlenet_results